In [1]:
import pickle  
import firebase_admin
import pandas as pd
from firebase_admin import db
from firebase_admin import credentials
import numpy as np
import joblib
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
live_data = 'liveData'

cred = credentials.Certificate("novalith-c49fb-firebase-adminsdk-fbsvc-ad5eb615de.json")
default_app = firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://novalith-c49fb-default-rtdb.firebaseio.com/'
})

ref_live = db.reference(live_data)

In [3]:
# Load the trained model
model = joblib.load('best_fetal_model_updated.pkl')

In [4]:
def inference_func_tabular(sample_json):
    df = pd.DataFrame([sample_json])
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)  # Handle NaN values by replacing them with 0
    
    if not hasattr(model, "predict"):
        raise TypeError("Loaded object is not a valid ML model. Please check the model file.")
    
    prediction = model.predict(df)[0] 
    categories = {0: 'Abnormal', 1: 'Normal'}
    return categories.get(int(prediction), 'Unknown')

In [5]:
def firebase_pipeline_tabular():
    all_data = ref_live.get()
    if not all_data:
        return
    
    keys_to_delete = []
    for data, value in all_data.items():
        if data == 'isNew' and value == "true":
            keys_to_delete.append('isNew')

    for data in keys_to_delete:
        del all_data[data]
        
        ecg = float(all_data.get('ecg', 0))
        blood_pressure = float(all_data.get('blood_pressure', 1))  # Avoid division by zero
        body_temp = float(all_data.get('body_temp', 0))
        heart_rate = float(all_data.get('heart_rate', 0))
        kicks_count = float(all_data.get('kicks_count', 0))
        
        sample_data = {
            'ECG': ecg,
            'Pressure': blood_pressure,
            'Temperature': body_temp,
            'BPM': heart_rate,
            'FetalCount': kicks_count,
            'ECG_Pressure_Ratio': ecg / blood_pressure if blood_pressure != 0 else 0
        }
        
        print(sample_data)
        ref_live.update({'isNew': "false"})
        status = inference_func_tabular(sample_data)
        print(status)
        ref_live.update({'Prediction': status})

In [ ]:
while True:
    firebase_pipeline_tabular()